In [68]:
import os 
from pathlib2 import Path

from ocrpy.io import DocumentReader
from ocrpy.parsers import TesseractTextOCR, AwsTextOCR

In [69]:
s3_img_path = "s3://rita-experiments/temp.png"
s3_pdf_path = "s3://rita-experiments/applsci-11-10639-v2.pdf"
local_img_path = "temp.png"

credentials = "credentials.env"

In [70]:
reader = DocumentReader(file=s3_pdf_path, credentials=credentials)

local_parser = TesseractTextOCR(reader)
aws_parser = AwsTextOCR(env_file=credentials, reader=reader)

In [12]:
local_parsed_data = local_parser.parse
aws_parsed_data = aws_parser.parse

In [41]:
from attr import define, field 
from uuid import uuid4
import json 
from dotenv import load_dotenv
from cloudpathlib import S3Client, GSClient, AnyPath

In [77]:
@define 
class DocumentWriter:
    data = field()
    file = field()
    credentials = field(default=None)

    @property
    def write(self):
        client = self._get_client(self.file)
        file_data = AnyPath(self.file, client=client)
        file_data.write_text(self.data)
        return None 

    def _get_client(self, file):
        if file.startswith("gs://") and self.credentials:
            client = GSClient(application_credentials=self.credentials)

        elif file.startswith("s3://") and self.credentials:
            load_dotenv(self.credentials)
            client = S3Client(aws_access_key_id=os.getenv(
                'aws_access_key_id'), aws_secret_access_key=os.getenv('aws_secret_access_key'))
        else:
            client = None

        return client

In [78]:
s3_destintion = "s3://rita-experiments/data_sample_1.json"
local_destination = "data_sample.json"
writer = DocumentWriter(file=s3_destintion, credentials=credentials, data=json.dumps(aws_parsed_data))

In [76]:
writer.write